### Assignment 5: Option Pricing using Black Scholes Formula
**By: Ashish Mathew**

In [ ]:
#!pip install pandas numpy scipy seaborn jupyter notebook yfinance

In [4]:
import pandas as pd
import numpy as np
from scipy.optimize import brentq
from scipy.stats import norm
import warnings
from datetime import datetime, timedelta
import yfinance as yf

warnings.filterwarnings('ignore')

In [60]:
def calc_annual_vol(ticker,start_dt,end_dt):
    # Pull price data from yfinance
    data = yf.download(tickers=ticker,start=start_dt,end=end_dt)
    prices = data['Close']

    daily_returns = prices.pct_change().dropna().values # Calculate Daily Return
    daily_vol = np.std(daily_returns) 
    annual_vol = daily_vol * np.sqrt(252) # Assuming trading year has 252 days
    last_price = prices.iloc[-1][0] # Latest close price

    return last_price, annual_vol

In [ ]:
def black_scholes(S, K, T, r, annual_vol, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * annual_vol**2) * T) / (annual_vol * np.sqrt(T))
    d2 = d1 - annual_vol * np.sqrt(T)

    if option_type.lower() == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    elif option_type.lower() == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    
    return price

In [57]:
def objective_function(sigma):
    return black_scholes(S,K,T,r,sigma,'call') - call_price

def bisection_implied_vol(call_price,S,K,T,r,precision=1e-8,max_iter=100):
    low_vol = 1e-3
    high_vol = 5.0

    for i in range(max_iter):
        mid_vol = (low_vol + high_vol) / 2
        price = black_scholes(S,K,T,r,mid_vol,'call')
    
        if abs(price - call_price) < precision:
            return mid_vol
        if price > call_price:
            high_vol = mid_vol
        else:
            low_vol = mid_vol

    return mid_vol

In [58]:
def calc_implied_volatility(call_price,S,K,T,r,initial_guess=0.2,max_iter=100,precision=1e-8):
    
    try:
        implied_vol = brentq(objective_function,1e-3,5.0,xtol=precision,maxiter=max_iter)
        return implied_vol
    except ValueError:
        return bisection_implied_vol(call_price,S,K,T,r,
                                     precision=precision,
                                     max_iter=max_iter)

In [80]:
def option_pricing(ticker,expiration_date,K,r):
    
    end_dt = datetime.today().date()
    start_dt = end_dt - timedelta(days=365) # 1 year lookback
    
    print(f"Pulling price data from {start_dt} to {end_dt} for {ticker}")

    S, sigma = calc_annual_vol(ticker,start_dt,end_dt)
    print(f"Last close price: ${S:.2f}")
    print(f"Calculated Volatility: {100*sigma:.2f}%")

    T = expiration_date - end_dt
    print(f"Time to maturity: {T.days} days")
    T = T.days / 365

    call_price = black_scholes(S,K,T,r,sigma,'call')
    put_price = black_scholes(S,K,T,r,sigma,'put')

    print(f"Call option premium: ${call_price:.2f}")
    print(f"Put option premium: ${put_price:.2f}")

    implied_vol = calc_implied_volatility(call_price,S,K,T,r)
    print(f"Implied Volatility: {100*implied_vol:.2f}%")

![alt text](./assets/JPM_call.png)
![alt text](./assets/JPM_put.png)

In [81]:
expiration_date = datetime(2025,8,8).date()
option_pricing('JPM',expiration_date,290,0.05)

Pulling price data from 2024-08-06 to 2025-08-06 for JPM


[*********************100%***********************]  1 of 1 completed

Last close price: $291.37
Calculated Volatility: 27.54%
Time to maturity: 2 days
Call option premium: $3.16
Put option premium: $1.71
Implied Volatility: 20.00%


Our calculated volatility of 27.5% is much higher than the implied volatility listed by Yahoo Finance of 25%. This higher volatility number leads to an overestimation of option premiums since the premium is directly proportional to the volatility of the underlying